In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

In [2]:
# Load the trained model
model = load_model('Model_Quality_Condition.h5')

In [3]:
# Process the data
df=pd.read_excel('MOOD TRAK.xlsx')
df_new=df.drop(columns=['Note','Duration'])
# Convert time columns to datetime
df_new['Time Stamp'] = pd.to_datetime(df_new['Time Stamp']).dt.date

In [4]:
# Define weights for each activity (higher weight means higher significance)
activity_weights = {
    'Sleep': 0.2,
    'Study': 0.1,
    'Work': 0.35,
    'Dating':0.05,
    'Self Care':0.05,
    'Traveling':0.05,
    'Entertainment':0.1,
    'Eating':0.05,
    'Workout':0.05
}
# Calculate overall day quality score for each day
df_new['Overall_Quality'] = df_new['Quality'] * df_new['Activities'].map(activity_weights)
daily_quality = df_new.groupby(['Time Stamp']).agg({'Overall_Quality': 'mean'}).reset_index()

In [5]:
X_new = daily_quality.drop(columns=['Time Stamp'])

In [6]:
# Make predictions
predictions = model.predict(X_new)

# Add the predicted classes to the new dataset
daily_quality['Predicted_Day_Condition'] = predictions.argmax(axis=1)

# Print or save the new dataset with predictions
daily_quality

4/4 [==============================] - 0s 6ms/step


,Time Stamp,Overall_Quality,Predicted_Day_Condition
0,2024-05-01,0.257143,3
1,2024-05-02,0.226923,3
2,2024-05-03,0.319231,2
3,2024-05-04,0.230000,3
4,2024-05-05,0.282143,3
...,...,...,...
93,2024-08-02,0.346154,2
94,2024-08-03,0.346154,2
95,2024-08-04,0.346154,2
96,2024-08-05,0.346154,2


In [7]:
# Create a new LabelEncoder instance
label_encoder = LabelEncoder()
original_class_labels = ['Bad', 'Excellent', 'Good', 'Okay','Terrible']
# Fit the LabelEncoder to the original class labels
label_encoder.fit(original_class_labels)

# Inverse transform the predicted classes to get the original class labels
daily_quality['Predicted_Day_Condition'] = label_encoder.inverse_transform(daily_quality['Predicted_Day_Condition'])


# Print or save the new dataset with predicted class labels
daily_quality

,Time Stamp,Overall_Quality,Predicted_Day_Condition
0,2024-05-01,0.257143,Okay
1,2024-05-02,0.226923,Okay
2,2024-05-03,0.319231,Good
3,2024-05-04,0.230000,Okay
4,2024-05-05,0.282143,Okay
...,...,...,...
93,2024-08-02,0.346154,Good
94,2024-08-03,0.346154,Good
95,2024-08-04,0.346154,Good
96,2024-08-05,0.346154,Good


In [8]:
# Define classification thresholds based on the overall day quality score
classification_thresholds = {
    'Terrible': 0.125000,
    'Bad': 0.200000,
    'Okay': 0.300000,
    'Good': 0.550000,
    'Excellent': 0.800000,
}
# Classify each day based on the overall quality score
def classify_day(quality_score):
    for category, threshold in classification_thresholds.items():
        if quality_score < threshold:
            return category

daily_quality['Real Category'] = daily_quality['Overall_Quality'].apply(classify_day)

daily_quality

,Time Stamp,Overall_Quality,Predicted_Day_Condition,Real Category
0,2024-05-01,0.257143,Okay,Okay
1,2024-05-02,0.226923,Okay,Okay
2,2024-05-03,0.319231,Good,Good
3,2024-05-04,0.230000,Okay,Okay
4,2024-05-05,0.282143,Okay,Okay
...,...,...,...,...
93,2024-08-02,0.346154,Good,Good
94,2024-08-03,0.346154,Good,Good
95,2024-08-04,0.346154,Good,Good
96,2024-08-05,0.346154,Good,Good


In [10]:
# Compare the predicted and real categories
daily_quality['Comparison'] = daily_quality['Predicted_Day_Condition'] == daily_quality['Real Category']
# Summary of comparison
correct_predictions = daily_quality['Comparison'].sum()
total_predictions = len(daily_quality)
accuracy = correct_predictions / total_predictions

print(f"Correct Predictions: {correct_predictions}/{total_predictions}")
print(f"Accuracy: {accuracy:.2%}")

Correct Predictions: 82/98
Accuracy: 83.67%


In [13]:
# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="Model_Quality_Condition.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Manually input data for inference
# Replace this with your own input data
input_data = np.array([[0.56875]], dtype=np.float32)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Post-process output data to obtain class labels
class_labels = np.argmax(output_data, axis=1)
# Define class labels
class_labels = ['Bad', 'Excellent', 'Good', 'Okay','Terrible']

# Post-process output data to obtain string class labels
class_indices = np.argmax(output_data, axis=1)
string_labels = [class_labels[i] for i in class_indices]

# Output string class labels
string_labels

['Excellent']